In [ ]:
from sklearn.cluster import KMeans
import h5py
import numpy as np  
import matplotlib.pyplot as plt
from sklearn import preprocessing
import scipy.io as sio
import sklearn.metrics as sm
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn_extensions.extreme_learning_machines.elm import GenELMClassifier

Factor = 1

os.chdir('/')
f = h5py.File('./Data.mat')
xyzData = {}
for k, v in f.items():
    xyzData[k] = np.array(v)
df = np.transpose(xyzData['Data'])

kmeans = KMeans(n_clusters=2, n_init = 10000).fit(df)

klabels = kmeans.predict(df)
print("labels using kmeans are:", klabels)
k_score = sm.silhouette_score(df, klabels, metric = 'euclidean', sample_size = len(df))
print("Silhouette score of kmeans method is:", k_score)

clustering = AgglomerativeClustering(n_clusters=2).fit(df)
Hilabels = clustering.labels_
print("labels using hierarchical clustering are:", Hilabels)
Hi_score = sm.silhouette_score(df, Hilabels, metric = 'euclidean', sample_size = len(df))
print("Silhouette score of hierarchical clustering method is:", Hi_score)

MotorSkill = np.repeat([1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1], Factor)
Flanders   = np.repeat([1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1], Factor)
Waterloo   = np.repeat([1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1], Factor)
Annett     = np.repeat([1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1], Factor)
Edinburgh  = np.repeat([1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1], Factor)

real= klabels
#print(real)
predicted= Flanders

list1 = np.arange(len(real))[real == 1]
list2 = np.arange(len(predicted))[np.transpose(predicted) == 1]
tp_R =len(list(set(list1).intersection(list2)))

list1 = np.arange(len(real))[real == 0]
list2 = np.arange(len(predicted))[np.transpose(predicted) == 0]
tp_L =  len(list(set(list1).intersection(list2)))

list1 = np.arange(len(real))[real == 0]
list2 = np.arange(len(predicted))[np.transpose(predicted) == 1]
fp_R = len(list(set(list1).intersection(list2)))

list1 = np.arange(len(real))[real == 1]
list2 = np.arange(len(predicted))[np.transpose(predicted) == 0]
fp_L = len(list(set(list1).intersection(list2)))

Precision_R = tp_R/(tp_R+fp_R)
Precision_L = tp_L/(tp_L+fp_L)

Recall_R = tp_R/(tp_R+fp_L)
Recall_L = tp_L/(tp_L+fp_R)

Accuracy = len(np.arange(len(real))[real == predicted])*100/len(MotorSkill)

print("Right Precision is:", Precision_R)
print("Left Precision is:", Precision_L)
print("Right Recall is:", Recall_R)
print("Left Recall is:", Recall_L)
print("Accuracy is:", Accuracy)

loo = LeaveOneOut()
myReal = np.array(Flanders)

SumAcc = 0
tpR = 0
tpL = 0
fpR = 0
fpL = 0
for train_index, test_index in loo.split(MotorSkill):
    df_train = df[train_index]
    df_test  = df[test_index]
    y_train  = myReal[train_index]
    y_test   = myReal[test_index]
    clf = LogisticRegression().fit(df_train, y_train) 
    MyRealLabel = y_test 
    MyPredictedLabel = clf.predict(df_test)
    SumAcc = SumAcc + (MyRealLabel == MyPredictedLabel)
    tpR = tpR + (MyRealLabel == 1 and MyPredictedLabel == 1)
    tpL = tpL + (MyRealLabel == 0 and MyPredictedLabel == 0)
    fpR = fpR + (MyRealLabel == 0 and MyPredictedLabel == 1)
    fpL = fpL + (MyRealLabel == 1 and MyPredictedLabel == 0)

RegMeanAcc = 100*SumAcc/len(MotorSkill)
PrecisionR = tpR/(tpR+fpR)
PrecisionL = tpL/(tpL+fpL)
RecallR = tpR/(tpR+fpL)
RecallL = tpL/(tpL+fpR)
print("Logistic Regression Accuracy is:", RegMeanAcc)
print('Right Precision is:', PrecisionR)
print('Left Precision is:', PrecisionL)
print('Right Recall is:', RecallR)
print('Left Recall is:', RecallL)    

SumAcc = 0
tpR = 0
tpL = 0
fpR = 0
fpL = 0
for train_index, test_index in loo.split(MotorSkill):
    df_train = df[train_index]
    df_test  = df[test_index]
    y_train  = myReal[train_index]
    y_test   = myReal[test_index]
    clf = SVC().fit(df_train, y_train)
    MyRealLabel = y_test 
    MyPredictedLabel = clf.predict(df_test)
    SumAcc = SumAcc + (MyRealLabel == MyPredictedLabel)
    tpR = tpR + (MyRealLabel == 1 and MyPredictedLabel == 1)
    tpL = tpL + (MyRealLabel == 0 and MyPredictedLabel == 0)
    fpR = fpR + (MyRealLabel == 0 and MyPredictedLabel == 1)
    fpL = fpL + (MyRealLabel == 1 and MyPredictedLabel == 0)

RegMeanAcc = 100*SumAcc/len(MotorSkill)
PrecisionR = tpR/(tpR+fpR)
PrecisionL = tpL/(tpL+fpL)
RecallR = tpR/(tpR+fpL)
RecallL = tpL/(tpL+fpR)
print("SVM Accuracy is:", RegMeanAcc)
print('Right Precision is:', PrecisionR)
print('Left Precision is:', PrecisionL)
print('Right Recall is:', RecallR)
print('Left Recall is:', RecallL)

SumAcc = 0
tpR = 0
tpL = 0
fpR = 0
fpL = 0
for train_index, test_index in loo.split(MotorSkill):
    df_train = df[train_index]
    df_test  = df[test_index]
    y_train  = myReal[train_index]
    y_test   = myReal[test_index]
    clf = SVC(kernel='linear').fit(df_train, y_train)
    MyRealLabel = y_test 
    MyPredictedLabel = clf.predict(df_test)
    SumAcc = SumAcc + (MyRealLabel == MyPredictedLabel)
    tpR = tpR + (MyRealLabel == 1 and MyPredictedLabel == 1)
    tpL = tpL + (MyRealLabel == 0 and MyPredictedLabel == 0)
    fpR = fpR + (MyRealLabel == 0 and MyPredictedLabel == 1)
    fpL = fpL + (MyRealLabel == 1 and MyPredictedLabel == 0)

RegMeanAcc = 100*SumAcc/len(MotorSkill)
PrecisionR = tpR/(tpR+fpR)
PrecisionL = tpL/(tpL+fpL)
RecallR = tpR/(tpR+fpL)
RecallL = tpL/(tpL+fpR)
print("SVM Linear Kernel Accuracy is:", RegMeanAcc)
print('Right Precision is:', PrecisionR)
print('Left Precision is:', PrecisionL)
print('Right Recall is:', RecallR)
print('Left Recall is:', RecallL)

SumAcc = 0
tpR = 0
tpL = 0
fpR = 0
fpL = 0
for train_index, test_index in loo.split(MotorSkill):
    df_train = df[train_index]
    df_test  = df[test_index]
    y_train  = myReal[train_index]
    y_test   = myReal[test_index]
    clf = KNeighborsClassifier(n_neighbors=10).fit(df_train, y_train)
    MyRealLabel = y_test 
    MyPredictedLabel = clf.predict(df_test)
    SumAcc = SumAcc + (MyRealLabel == MyPredictedLabel)
    tpR = tpR + (MyRealLabel == 1 and MyPredictedLabel == 1)
    tpL = tpL + (MyRealLabel == 0 and MyPredictedLabel == 0)
    fpR = fpR + (MyRealLabel == 0 and MyPredictedLabel == 1)
    fpL = fpL + (MyRealLabel == 1 and MyPredictedLabel == 0)

RegMeanAcc = 100*SumAcc/len(MotorSkill)
PrecisionR = tpR/(tpR+fpR)
PrecisionL = tpL/(tpL+fpL)
RecallR = tpR/(tpR+fpL)
RecallL = tpL/(tpL+fpR)
print("K-Neighbours Accuracy is:", RegMeanAcc)
print('Right Precision is:', PrecisionR)
print('Left Precision is:', PrecisionL)
print('Right Recall is:', RecallR)
print('Left Recall is:', RecallL)

SumAcc = 0
tpR = 0
tpL = 0
fpR = 0
fpL = 0
for train_index, test_index in loo.split(MotorSkill):
    df_train = df[train_index]
    df_test  = df[test_index]
    y_train  = myReal[train_index]
    y_test   = myReal[test_index]
    clf = GenELMClassifier().fit(df_train, y_train)
    MyRealLabel = y_test 
    MyPredictedLabel = clf.predict(df_test)
    SumAcc = SumAcc + (MyRealLabel == MyPredictedLabel)
    tpR = tpR + (MyRealLabel == 1 and MyPredictedLabel == 1)
    tpL = tpL + (MyRealLabel == 0 and MyPredictedLabel == 0)
    fpR = fpR + (MyRealLabel == 0 and MyPredictedLabel == 1)
    fpL = fpL + (MyRealLabel == 1 and MyPredictedLabel == 0)

RegMeanAcc = 100*SumAcc/len(MotorSkill)
PrecisionR = tpR/(tpR+fpR)
PrecisionL = tpL/(tpL+fpL)
RecallR = tpR/(tpR+fpL)
RecallL = tpL/(tpL+fpR)
print("ELM Accuracy is:", RegMeanAcc)
print('Right Precision is:', PrecisionR)
print('Left Precision is:', PrecisionL)
print('Right Recall is:', RecallR)
print('Left Recall is:', RecallL)